# What is Cython?
### Cython is a programming language that aims to be a superset of the Python programming language, designed to give C-like performance with code that is written mostly in Python with optional additional C-inspired syntax.

### Cython is a compiled language that is typically used to generate CPython extension modules. Annotated Python-like code is compiled to C or C++ then automatically wrapped in interface code, producing extension modules that can be loaded and used by regular Python code using the import statement, but with significantly less computational overhead at run time. Cython also facilitates wrapping independent C or C++ code into python-importable modules.

### First of all we need to install cython using ```pip install cython ``` and once installed we can load cython session using ```%load_ext cython```

In [1]:
!pip install cython

In [5]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


### Let's define here our benchmark function

In [3]:
from scipy.stats import norm
import numpy as np

def python_price_european_option(sigma,S0, K, CP , T,t=0): #CP=1 for call CP=-1 for put
    tau = T - t
    sigmtau = sigma*np.sqrt(tau)
    k = np.log(K/S0)
    dp = -k / sigmtau + 0.5*sigmtau
    dm = dp - sigmtau
    return S0*(CP*norm.cdf(CP*dp) - CP*np.exp(k)*norm.cdf(CP*dm))
%timeit python_price_european_option(sigma=0.2,S0=100,K=90,CP=-1,T=0.25)

216 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### To define a cython function or functions we need to add  ```%%cython``` in the beginning of the cell to let the compiler know we will be using cython. Also we use the syntax ```cpdef``` instead of  ```def```  to tell the compiler that this is both a python and c function. Cython allows to use ```cdef``` for a pure c function (which means input and outputs must be c accepted types)

In [4]:
%%cython
from scipy.stats import norm
import numpy as np
cpdef c_naive_price_european_option(sigma,S0, K, CP, T,t=0):
    tau = T - t
    sigmtau = sigma*np.sqrt(tau)
    k = np.log(K/S0)
    dp = -k / sigmtau + 0.5*sigmtau
    dm = dp - sigmtau
    return S0*(CP*norm.cdf(CP*dp) - CP*np.exp(k)*norm.cdf(CP*dm))



DistutilsPlatformError: Unable to find vcvarsall.bat

In [ ]:
%timeit c_naive_price_european_option(sigma=0.2,S0=100,K=90,CP=-1,T=0.25)

###  Well not very spectacular right? To help us detect the parts where we can improve we can use the flag ```%% cython -a```

In [ ]:
%%cython -a 
from scipy.stats import norm
import numpy as np
cpdef c_naive_price_european_option(sigma,S0, K, CP, T,t=0):
    tau = T - t
    sigmtau = sigma*np.sqrt(tau)
    k = np.log(K/S0)
    dp = -k / sigmtau + 0.5*sigmtau
    dm = dp - sigmtau
    return S0*(CP*norm.cdf(CP*dp) - CP*np.exp(k)*norm.cdf(CP*dm))


### Yellow line essentially tell us which part of the code are being run in pure python, as you can see is all of it so we are of course not making any improvement yet. The first thing to do to improve our code is to define the types of our variables, as you would do in a compiled language

In [ ]:
%%cython -a
from scipy.stats import norm
import numpy as np
cpdef double c_with_types_price_european_option(double sigma, double S0, double K, double CP, double T,double t=0):
    cdef double tau = T - t
    cdef double sigmtau = sigma*np.sqrt(tau)
    cdef double k = np.log(K/S0)
    cdef double dp = -k / sigmtau + 0.5*sigmtau
    cdef double dm = dp - sigmtau
    return S0*(CP*norm.cdf(CP*dp) - CP*np.exp(k)*norm.cdf(CP*dm))

In [ ]:
%timeit c_with_types_price_european_option(sigma=0.2,S0=100,K=90,CP=-1,T=0.25)

###  Still quite a few lines to go. As you can see, we are still using ```np.sqrt``` , ```np.log``` and ```np.exp```. Is is preferred to change this to c native functions

In [ ]:
%%cython -a
from scipy.stats import norm
from libc.math cimport  sqrt,log,exp
cpdef double c_v1_price_european_option(double sigma, double S0, double K, double CP, double T,double t=0):
    cdef double tau = T - t
    cdef double sigmtau = sigma*sqrt(tau)
    cdef double k = log(K/S0)
    cdef double dp = -k / sigmtau + 0.5*sigmtau
    cdef double dm = dp - sigmtau
    return S0*(CP*norm.cdf(CP*dp) - CP*exp(k)*norm.cdf(CP*dm))

In [ ]:
%timeit c_v1_price_european_option(sigma=0.2,S0=100,K=90,CP=-1,T=0.25)

### we see that we still have some ligh yellow lines, this is because python checks for zero divisions we can improve this by using the decorator  ```@cython.cdivision(True)```

In [ ]:
%%cython -a
cimport cython
from scipy.stats import norm
from libc.math cimport  sqrt,log,exp
@cython.cdivision(True)
cpdef double c_v2_price_european_option(double sigma, double S0, double K, double CP, double T,double t=0):
    cdef double tau = T - t
    cdef double sigmtau = sigma*sqrt(tau)
    cdef double k = log(K/S0)
    cdef double dp = -k / sigmtau + 0.5*sigmtau
    cdef double dm = dp - sigmtau
    return S0*(CP*norm.cdf(CP*dp) - CP*exp(k)*norm.cdf(CP*dm))

In [ ]:
%timeit c_v2_price_european_option(sigma=0.2,S0=100,K=90,CP=-1,T=0.25)

### As you may have already guessed the main bottlenec here is the gaussian cdf 
$$\Phi(x) =\frac{1}{\sqrt{2\pi}}\int_{-\infty}^x e^\tfrac{-t^2}{2}\,dt = \frac{1}{2} \left[1+\operatorname{erf}\left(\frac{x}{\sqrt{2}}\right)\right]$$
lucky for us, the error function ```erf()``` is build in in the standard c library

In [ ]:
%%cython -a
cimport cython
from libc.math cimport  sqrt,log,exp,erf

@cython.cdivision(True)
cdef double gaussian_cdf(double x):
    return 0.5*(1+erf(x/sqrt(2)))

@cython.cdivision(True)
cpdef double c_v3_price_european_option(double sigma, double S0, double K, double CP, double T,double t=0):
    cdef double tau = T - t
    cdef double sigmtau = sigma*sqrt(tau)
    cdef double k = log(K/S0)
    cdef double dp = -k / sigmtau + 0.5*sigmtau
    cdef double dm = dp - sigmtau
    return S0*(CP*gaussian_cdf(CP*dp) - CP*exp(k)*gaussian_cdf(CP*dm))

In [ ]:
%timeit c_v3_price_european_option(sigma=0.2,S0=100,K=90,CP=-1,T=0.25)

### Well that is certainly some speed improvement!  Now we can further improve some stuff like constants. We have th value $\frac{1}{\sqrt{2}}$ being computed numerically but we can set it as a global constant. Furthermore, we can add a bunch of decorators:

1. ```@cython.boundscheck(False)``` : Boundscheck is a security check that you are accessing indices inside the bounds of the vectors.

2. ```@cython.wraparound(False)``` :  Deactivate negative indexing on arrays

3. ```@cython.nonecheck(False)``` :  Deactivate checking for none

## And we can define our python function as a pure c function as there is no dependency on python

In [ ]:
%%cython -a 
cimport cython
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double gaussian_cdf(double x):
    return 0.5*(1+erf(x*ONE_OVER_SQRT_TWO))

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cpdef double c_v4_price_european_option(double sigma, double S0, double K, double CP, double T,double t=0):
    cdef double tau = T - t
    cdef double sigmtau = sigma*sqrt(tau)
    cdef double k = log(K/S0)
    cdef double dp = -k / sigmtau + 0.5*sigmtau
    cdef double dm = dp - sigmtau
    return S0*(CP*gaussian_cdf(CP*dp) - CP*exp(k)*gaussian_cdf(CP*dm))

In [ ]:
%timeit c_v4_price_european_option(sigma=0.2,S0=100,K=90,CP=-1,T=0.25)

### As you can see not much improvement as we are not using vectors. The last thing left to do is to deactivate the GIL (Global Interpreter Lock) to do this we can use the syntax ```nogil``` after defining the function. Furthermore we can add compiler arguments like ```%%cython -a --compile-args=-O3``` to improve perfomance, but we should have achieved maximum or close tomaximum performance

In [ ]:
%%cython -a --compile-args=-O3
cimport cython
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double gaussian_cdf(double x) nogil:
    return 0.5*(1+erf(x*ONE_OVER_SQRT_TWO))

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cpdef double c_v5_price_european_option(double sigma, double S0, double K, double CP, double T,double t=0) nogil:
    cdef double tau = T - t
    cdef double sigmtau = sigma*sqrt(tau)
    cdef double k = log(K/S0)
    cdef double dp = -k / sigmtau + 0.5*sigmtau
    cdef double dm = dp - sigmtau
    return S0*(CP*gaussian_cdf(CP*dp) - CP*exp(k)*gaussian_cdf(CP*dm))

In [ ]:
%timeit c_v5_price_european_option(sigma=0.2,S0=100,K=90,CP=-1,T=0.25)

# Arrays and cython
### So far we have seen how to construct a function whose arguments are 1 dimensional. How about performing operations on an array? Let's imagine we want to compute the Black-Scholes price for a bunc of strikes

In [ ]:
import numpy as np
strikes=np.linspace(25,300,1000000)
print(strikes)

In [ ]:
c_v5_price_european_option(sigma=0.2,S0=100,K=strikes,CP=-1,T=0.25)

### The function obviously does not accept the array as input and we are left with no choice but to use some sort of list comprehension

In [ ]:
%timeit [c_v5_price_european_option(sigma=0.2,S0=100,K=strike,CP=-1,T=0.25) for strike in strikes]

# Cython ndarrays

### cython ndarrays allow to take numpy arrays as input. To do so we need to ```cimport numpy as cnp``` and define ```cnp.ndarray[cnp.data_type, ndim=number_dimensions]```. However the GIL is necessary to allocate necessary buffers. Luckily Cython allows to switch off the GIL on a block of code so long as all variables in this block are well defined C types, to do so we use the syntax ```with nogil:``` see below 

In [ ]:
%%cython --compile-args=-O3
cimport cython
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476
import numpy as np
cimport numpy as cnp

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double gaussian_cdf(double x) nogil:
    return 0.5*(1+erf(x*ONE_OVER_SQRT_TWO))

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cpdef cnp.ndarray[cnp.double_t, ndim=1] c_v6_price_european_option(double sigma, double S0, cnp.ndarray[cnp.double_t, ndim=1] K, double CP, double T,double t=0):
    cdef double tau = T - t
    cdef int size=len(K)
    cdef double sigmtau = sigma*sqrt(tau)
    cdef double k,dp,dm
    cdef cnp.ndarray[cnp.double_t, ndim=1] result=np.zeros(size)
        
    with nogil:
        for i in range(size):
            k = log(K[i]/S0)
            dp = -k / sigmtau + 0.5*sigmtau
            dm = dp - sigmtau
            result[i]=S0*(CP*gaussian_cdf(CP*dp) - CP*exp(k)*gaussian_cdf(CP*dm))
        
    return result

In [ ]:
%timeit c_v6_price_european_option(sigma=0.2,S0=100,K=strikes,CP=-1,T=0.25)

In [ ]:
c_v6_price_european_option(sigma=0.2,S0=100,K=strikes,CP=-1,T=0.25)

# Cython memoryviews

### Cython also allows to remove completely the use of python to handle numpy arrays using memoryviews which are similar to C arrays. The syntax to define a memoryview is ```double[:] my_array``` and we can access their size without the GIL using ```.shape```. Note however, that if a new memoriview object needs to be created for an output, then the GIL will be called to make the object available

In [ ]:
%%cython -a --compile-args=-O3
cimport cython
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476
from libc.stdlib cimport malloc, free

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double gaussian_cdf(double x) nogil:
    return 0.5*(1+erf(x*ONE_OVER_SQRT_TWO))

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cpdef double[:] c_v7_price_european_option(double sigma, double S0, double[:] K, double CP, double T,double t=0):
    cdef double tau = T - t
    cdef double sigmtau = sigma*sqrt(tau)
    cdef double k,dp,dm
    cdef int i
    cdef int N=K.shape[0]
    cdef double *res = <double *> malloc(N * sizeof(double))
    cdef double[:] result = <double[:N]>res
    

    for i in range(N):
        k = log(K[i]/S0)
        dp = -k / sigmtau + 0.5*sigmtau
        dm = dp - sigmtau
        result[i]=S0*(CP*gaussian_cdf(CP*dp) - CP*exp(k)*gaussian_cdf(CP*dm))
        
    return result

In [ ]:
%timeit c_v7_price_european_option(sigma=0.2,S0=100,K=strikes,CP=-1,T=0.25)

### Note that the return type is a memoriview, which can be accessed element by element

In [ ]:
memoryview_array=c_v7_price_european_option(sigma=0.2,S0=100,K=strikes,CP=-1,T=0.25)

In [ ]:
memoryview_array

In [ ]:
print("5000th element",memoryview_array[5000])
print("length is :",len(memoryview_array))

### or casted into a numpy array

In [ ]:
np.asarray(memoryview_array)

# Note:
## Using ```malloc``` allocates memory in the RAM memory of your computer. To free the memory one needs to use ```free```. In this case, since the python object has been created with malloc, the memory will be allocated as long as the object lives e.g. until the kernel is shutdown

In [ ]:
%%cython -a --compile-args=-O3
cimport cython
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476
from libc.stdlib cimport malloc, free
from cpython.mem cimport PyMem_Malloc, PyMem_Realloc, PyMem_Free

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cpdef free_memory(double[:] data):
    PyMem_Free(&data[0])

In [ ]:
%timeit free_memory(c_v7_price_european_option(sigma=0.2,S0=100,K=strikes,CP=-1,T=0.25))

## If you check your memory you will see that it remains stable here

## This gives similar peformance, however if we want to obtain maximum performance we can pass the result array directly to the function so that no gil will be required, or we could transform K variable into our solution

In [ ]:
%%cython -a --compile-args=-O3
cimport cython
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476
from libc.stdlib cimport malloc, free

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double gaussian_cdf(double x) nogil:
    return 0.5*(1+erf(x*ONE_OVER_SQRT_TWO))

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cpdef void c_v8_price_european_option(double sigma, double S0, double[::1] K, double CP, double T,double t=0)nogil:
    cdef double tau = T - t
    cdef double sigmtau = sigma*sqrt(tau)
    cdef double k,dp,dm
    cdef int i
    cdef int N=K.shape[0]   

    for i in range(N):
        k = log(K[i]/S0)
        dp = -k / sigmtau + 0.5*sigmtau
        dm = dp - sigmtau
        K[i]=S0*(CP*gaussian_cdf(CP*dp) - CP*exp(k)*gaussian_cdf(CP*dm))
        

In [ ]:
%timeit c_v8_price_european_option(sigma=0.2,S0=100,K=strikes,CP=-1,T=0.25)

### Note that memoryviews are passed by reference, hence any change we make to the inputs will be permanent!

In [ ]:
strikes=np.linspace(50,150,100)
c_v8_price_european_option(sigma=0.2,S0=100,K=strikes,CP=-1,T=0.25)
print(strikes)

## A last performance remark! 
### Cython allows to do parallel for loops e.g. use all your computer cores to perform the loop. The syntax is very easy using ```for i in prange()```  and one needs to ```from cython.parallel import prange``` and also add the compiler flags for ```openmp``` which is a multiprocessing library


In [ ]:
%%cython -c=-fopenmp --link-args=-fopenmp
cimport cython
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476
from libc.stdlib cimport malloc, free
from cython.parallel import prange

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double gaussian_cdf(double x) nogil:
    return 0.5*(1+erf(x*ONE_OVER_SQRT_TWO))

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cpdef void c_parallel_price_european_option(double sigma, double S0, double[:] K, double CP, double T,double t=0)nogil:
    cdef double tau = T - t
    cdef double sigmtau = sigma*sqrt(tau)
    cdef double k,dp,dm
    cdef int i
    cdef int N=K.shape[0]   

    for i in prange(N,nogil=True,num_threads=4):
        k = log(K[i]/S0)
        dp = -k / sigmtau + 0.5*sigmtau
        dm = dp - sigmtau
        K[i]=S0*(CP*gaussian_cdf(CP*dp) - CP*exp(k)*gaussian_cdf(CP*dm))
        

In [ ]:
strikes=np.linspace(25,300,1000000)
%timeit c_parallel_price_european_option(sigma=0.2,S0=100,K=strikes,CP=-1,T=0.25)

### With 4 threads we see roughly 4x speed improvement

# A word in contiguous arrays:

### NUMPY arrays are NOT necessarilly contiguous!!! This means that if we use a C algorithm that requires contiguous arrays it will return unexpected values. There are two ways around this:

### 1. Use  ```np.ascontiguousarray ``` to cast a numpy array into a contguous version
### 2. In cython use ```[::1]``` intead of ```[:]``` to let cython know that a contiguous arrray is required

# Let's take an example on when things can go wrong. BLAS (Basic Linear Algebra Subprogram) is a highly efficient native library (C/Fortran) to perform basic linear algebra operations.

# As such it heavily uses the property of input arrayis being contiguois to leverage execution. We will look at the case of the dot product

In [ ]:
%reload_ext cython

In [ ]:
%%cython -a --compile-args=-O3 -l=blas
cimport cython 
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476
from libc.stdlib cimport malloc, free
cdef extern from '/usr/include/x86_64-linux-gnu/cblas.h' nogil:
    double cblas_ddot(int N,double* X, int incX,double* Y, int incY)
@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False) 
cpdef double  dot_product(double[:] x,double[:] y)nogil :
    return cblas_ddot(x.shape[0],&x[0],1,&y[0],1)
    

In [ ]:
import numpy as np
x=np.arange(100000,dtype=np.double)
y=np.arange(100000,dtype=np.double)
dot_product(x,y)

In [ ]:
%timeit dot_product(x,y)

In [ ]:
np.dot(x,y)

In [ ]:
%timeit np.dot(x,y)

### Curious note: We see that numpy's dot product has a fantastic performance, due to the fact thatit uses highly efficient Fortran/C subroutines in the backend. However, the benefit of using BLAS in this case is that it does not require the GIL

## Now let's consider the same dot product, using only even elements

In [ ]:
dot_product(x[::2],y[::2])

In [ ]:
np.dot(x[::2],y[::2])

## As you can see the results don't agree anymore. A solution is to cast to a contiguous array

In [ ]:
dot_product(np.ascontiguousarray(x[::2]),np.ascontiguousarray(y[::2]))

## We can also add the ```[::1]``` syntax to prevent us from making that mistake 

In [ ]:
%%cython -a --compile-args=-O3 -l=blas
cimport cython 
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476
from libc.stdlib cimport malloc, free
cdef extern from '/usr/include/x86_64-linux-gnu/cblas.h' nogil:
    double cblas_ddot(int N,double* X, int incX,double* Y, int incY)
@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False) 
cpdef double  dot_product(double[::1] x,double[::1] y)nogil :
    return cblas_ddot(x.shape[0],&x[0],1,&y[0],1)
    

In [ ]:
dot_product(x[::2],y[::2])

### No longer alows us to input non-contiguous arrays

# Cython classes
### Cython also allows to create classes. The syntax is similar to what we saw last session, but cython does allow to handle private attributes, in fact all attributes are private by default and not accesible by Python



In [ ]:
%%cython -a --compile-args=-O3
cimport cython
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476
from libc.stdlib cimport malloc, free

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double gaussian_cdf(double x) nogil:
    return 0.5*(1+erf(x*ONE_OVER_SQRT_TWO))

cdef class Black_Scholes:
    cdef double sigma
    def __init__(self, sigma):
        self.sigma=sigma
        
    cpdef double get_sigma(self):
        return self.sigma
    @cython.cdivision(True)
    @cython.boundscheck(False)
    @cython.wraparound(False)
    @cython.nonecheck(False)
    cpdef double european_option(self, double S0, double K, double CP, double T,double t=0):
        cdef double tau = T - t
        cdef double sigmtau = self.sigma*sqrt(tau)
        cdef double k = log(K/S0)
        cdef double dp = -k / sigmtau + 0.5*sigmtau
        cdef double dm = dp - sigmtau
        return S0*(CP*gaussian_cdf(CP*dp) - CP*exp(k)*gaussian_cdf(CP*dm))
    

In [ ]:
BS=Black_Scholes(0.2)
%timeit BS.european_option(S0=100,K=90,CP=-1,T=0.25)

In [ ]:
BS.sigma

In [ ]:
BS.get_sigma()

# Creating Modules
### So far all we have created run on the notebook but is nowhere created. We can create a module using the syntax ```cython_pyximport modulename``` , which will create a ```modulename.pyx``` file. The ```.pyx``` makes reference to the fact that this is a cython script rather than a python one

In [ ]:
%%cython_pyximport BS_module

cimport cython
from libc.math cimport  sqrt,log,exp,erf
cdef double ONE_OVER_SQRT_TWO=0.7071067811865476

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double gaussian_cdf(double x) nogil:
    return 0.5*(1+erf(x*ONE_OVER_SQRT_TWO))

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cpdef double module_price_european_option(double sigma, double S0, double K, double CP, double T,double t=0) nogil:
    cdef double tau = T - t
    cdef double sigmtau = sigma*sqrt(tau)
    cdef double k = log(K/S0)
    cdef double dp = -k / sigmtau + 0.5*sigmtau
    cdef double dm = dp - sigmtau
    return S0*(CP*gaussian_cdf(CP*dp) - CP*exp(k)*gaussian_cdf(CP*dm))

### We can then import our module using ```import pyximport; pyximport.install()``` which will know that the module we are trying to import is a Cython one

In [ ]:
import pyximport; pyximport.install()
import BS_module # name of .pyx file

In [ ]:
BS_module.module_price_european_option(sigma=0.2,S0=100,K=90,CP=-1,T=0.25)

# Importing c/c++ source code: A financial example with Implied Volatility
### The full power of cython comes to life when we are able to wrap c libraries or functions. We will present a example here. Let's be rational https://jaeckel.000webhostapp.com/LetsBeRational.pdf by Peter Jackel is one of the most influencial papers in the efficient computation of Implied Volatility and is a industry standard. Albeit efficient, the code is somewhat involved but Jackel was kind enough to release the code for the implementation in cpp (you can find it in http://jaeckel.org/). What we will do here is to create a wrapper of this library with cython so that we can use it in Python.

### SO far we have hidden how Cython manages code. What happens in reality is that the .pyx script is compiled into a .c/cpp code and the further compiled to be available to python. The examples we have discussed before work just fine as these are self-contained and do not involve external libraries. However, in general the way to go with Cython is to first create .pyx script and then compile it

### The function we want to import is located in  ```./LetsBeRational/erf_cody.h``` header file.  Cython allows to import external functions using the syntax 

```cdef extern from "path_to_file nogil:
    double function_to_import(args)```

### see and example below

### Note that this won't work in jupyter notebook so we need to copy paste it into a .pyx file

### Next we will create a ```setup.py``` file to create a installation/compilation script with the right directives to be able to use the modules. The contentents of this script should be the following

In [ ]:
!python3 setup.py build_ext --inplace

In [ ]:
import IV_module

In [ ]:
price=BS_module.module_price_european_option(sigma=0.45,S0=100,K=90,CP=-1,T=0.25)

In [ ]:
IV_module.implied_volatility(price=price, F=100, K=90, T=0.25, q=-1)

In [ ]:
%timeit IV_module.implied_volatility(price=price, F=100, K=90, T=0.25, q=-1)

## Voilà!! we have just created a state of the art IV calculation for your Python projects

# Very short note on Numba

### Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

### You don't need to replace the Python interpreter, run a separate compilation step, or even have a C/C++ compiler installed. Just apply one of the Numba decorators to your Python function, and Numba does the rest.

### Note that numba usually only interacts with Python and Numpy, so libraries like Scipy cannot be used in conjuction with Numba

In [ ]:
import math
import numpy as np
from numba import jit

@jit
def normcdf(d):
    A1 = 0.31938153
    A2 = -0.356563782
    A3 = 1.781477937
    A4 = -1.821255978
    A5 = 1.330274429
    RSQRT2PI = 0.39894228040143267793994605993438
    K = 1.0 / (1.0 + 0.2316419 * math.fabs(d))
    ret_val = (RSQRT2PI * math.exp(-0.5 * d * d) *
               (K * (A1 + K * (A2 + K * (A3 + K * (A4 + K * A5))))))
    if d > 0:
        ret_val = 1.0 - ret_val
    return ret_val

@jit
def numba_price_european_option(sigma,S0, K, CP, T,t=0):
    tau = T - t
    sigmtau = sigma*np.sqrt(tau)
    k = np.log(K/S0)
    dp = -k / sigmtau + 0.5*sigmtau
    dm = dp - sigmtau
    return S0*(CP*normcdf(CP*dp) - CP*np.exp(k)*normcdf(CP*dm))

In [ ]:
%timeit numba_price_european_option(sigma=0.2,S0=100,K=90,CP=-1,T=0.25)